In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
!pip install stable-baselines3[extra]

In [3]:
import numpy as np
# Import the game
import gym_super_mario_bros
# Import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [4]:
from gym.wrappers import FrameStack, GrayScaleObservation

from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

from matplotlib import pyplot as plt

In [5]:
# Create base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplify Controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# Format it for the AI to understand
env = DummyVecEnv([lambda: env])
# Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [6]:
# Import os file for path management
import os
# import PPO for algos
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.00005, 
            n_steps=512) 

In [ ]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=100000, callback=callback)

In [ ]:
model.save('thisisatestmodel')

In [7]:
model = PPO.load('./train/best_model_200000')

In [8]:
state = env.reset()

In [9]:
# Input/X Collector, picks the farthest iteration that Mario can make it and upload to a .csv file to give to server
# record the data during the learning process
mario_positions = []
actions = []


for i in range(1000):
    # take an action using the learned model
    action, _states = model.predict(state)
    state, reward, done, info = env.step(action)

    # record Mario's position, and the taken actions
    mario_x_position = info[0]['x_pos']
    mario_positions.append(mario_x_position)
    actions.append(action)

# find the step at which Mario got the farthest
farthest_step = np.argmax(mario_positions)
farthest_position = mario_positions[farthest_step]
farthest_action = actions[farthest_step]



print(f"Mario's farthest position: {farthest_position} at step {farthest_step}")
print(f"Action taken at farthest position: {farthest_action}")

C:\Users\owenf\anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Mario's farthest position: 315 at step 215
Action taken at farthest position: [1]


In [11]:
# Create a directory to store the CSV file
if not os.path.exists("data"):
    os.makedirs("data")

# Open the CSV file in write mode
with open(os.path.join("data", "data.csv"), "w") as file:
    # Write the variable values as a row in the CSV file
    file.write(f"{farthest_step},{farthest_action}")


In [ ]:
# Start the game 
state = env.reset()
# Loop through the game
counter = 0
while True: 
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()